In [2]:
import tensorflow as tf
import numpy as np
import os
import pathlib

# Step 1: Prepare the data
data_dir = 'path/to/oxford_iiit_pet_dataset'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [4]:
import pandas as pd

df = pd.read_csv('data.csv')

df['image'][0]



data = df.copy(deep=True)

# preprocess image data
# data['image'] = data['image'].apply(lambda x: np.array(Image.fromarray(x).resize((224, 224))))
# data['image'] = data['image'].apply(lambda x: x / 255.0)

# one-hot encode label
# data['label'] = data['label'].apply(lambda x: 1 if x == 'dog' else 0)
data = pd.get_dummies(data, columns=['label'])
# one-hot encode breed
data = pd.get_dummies(data, columns=['breed'])

In [5]:
# split data into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

# split train into train and validation
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [6]:
# add data/images/ to the filenames
train['filename'] = 'data/images/' + train['filename']
val['filename'] = 'data/images/' + val['filename']

In [7]:
# convert label_0 and label_1 to label_cat and label_dog
train = train.rename(columns={'label_0': 'label_cat', 'label_1': 'label_dog'})
val = val.rename(columns={'label_0': 'label_cat', 'label_1': 'label_dog'})

In [ ]:
# train the model
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='filename',
    y_col=['label_cat', 'label_dog'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    val,
    x_col='filename',
    y_col=['label_cat', 'label_dog'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

In [ ]:
base_models = [
    tf.keras.applications.VGG16(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.applications.ResNet50(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.applications.InceptionV3(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
]

# Step 3: Fine-tune the models
for model in base_models:
    for layer in model.layers:
        layer.trainable = False
    model.summary()
    x = model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(2, activation='softmax')(x)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    model.fit(train_ds, validation_data=val_ds, epochs=10)
    model.fit(train_generator, epochs=4, validation_data=val_generator)

# Step 4: Combine the predictions
test_images = []
for images, labels in test_ds:
    test_images.append(images.numpy())
test_images = np.concatenate(test_images)
preds = []
for model in base_models:
    preds.append(model.predict(test_images))
preds = np.concatenate(preds, axis=1)
ensemble_preds = np.argmax(np.mean(preds, axis=1), axis=1)

# Step 5: Evaluate the ensemble model
ensemble_acc = np.mean(ensemble_preds == test_ds.labels)
print('Ensemble accuracy:', ensemble_acc)

In [ ]:
def create_model(base_model):
    base_model.trainable = True
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [ ]:
batch_size = 32
epochs = 20
def fit_model(model):
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                      steps_per_epoch=len(total_train)//batch_size, 
                        epochs=epochs, 
                        validation_data=(X_test, y_test), 
                        validation_steps=len(total_val)//batch_size)
    return history
IMG_SHAPE = (224, 224, 3)

base_model1 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model2 = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model3 = tf.keras.applications.Xception(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

model1 = create_model(base_model1)
model2 = create_model(base_model2)
model3 = create_model(base_model3)

history1 = fit_model(model1)
model1.save('models/model1.h5')
history2 = fit_model(model2)
model2.save('models/model2.h5')
history3 = fit_model(model3)
model3.save('models/model3.h5')

In [ ]:
def load_all_models():
    all_models = []
    model_names = ['model1.h5', 'model2.h5', 'model3.h5']
    for model_name in model_names:
        filename = os.path.join('models', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models
    
models = load_all_models()
for i, model in enumerate(models):
    for layer in model.layers:
        layer.trainable = False

In [ ]:
ensemble_visible = [model.input for model in models]
ensemble_outputs = [model.output for model in models]
merge = tf.keras.layers.concatenate(ensemble_outputs)
merge = tf.keras.layers.Dense(10, activation='relu')(merge)
output = tf.keras.layers.Dense(1, activation='sigmoid')(merge)
model = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
X_train = [X_train for _ in range(len(model.input))]
X_test = [X_test for _ in range(len(model.input))]

In [ ]:
history = model.fit(X, y_train,
                    batch_size=batch_size,
                    steps_per_epoch=len(total_train) // batch_size,
                    epochs=epochs, 
                    validation_data=(X_1, y_test),
                    validation_steps=len(total_val) // batch_size)